In [9]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pickle

import warnings
warnings.filterwarnings('ignore')

In [10]:
def minmax(df: pd.DataFrame):
    minmax = MinMaxScaler().fit(df)
    X_norm_m = minmax.transform(df)
    X_norm_m = pd.DataFrame(X_norm_m, columns=df.columns)
    return X_norm_m


def onehot(df: pd.DataFrame):
    encoder = OneHotEncoder(drop='first').fit(pd.DataFrame(df))
    encoded = encoder.transform(pd.DataFrame(df)).toarray()
    cols = encoder.get_feature_names_out(input_features=df.columns)
    cols = list(map(normit, cols))  
    onehot_encoded_cats = pd.DataFrame(encoded, columns=cols)
    return onehot_encoded_cats


def normit(x: str):
    return x.replace(' ', '_').replace('/', '_').lower()


def get_height(x: str):
    feet, inch = x.split("'")
    inch = inch.rstrip('"')
    return int(feet)*30.48 + int(inch)*2.54


def get_value(x):
    x = str(x)
    if x[-1] == 'M':
        x = x[1:-1]
        return 1000000 * float(x)
    elif x[-1] == 'K':
        x = x[1:-1]
        return 1000 * float(x)
    else:
        x = x[1:]
        return float(x)


def get_stars(x):
    return int(x[0])

# commented lines are from initial model
def clean_fifa_df(df: pd.DataFrame):
    # dropped = df[['Age', 'BP', 'Height', 'Weight', 'foot', 'Growth', 'Value', 'Wage', 'Release Clause', 'Attacking', 
    # 'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 
    # 'Long Passing', 'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Power', 
    # 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning', 
    # 'Vision', 'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving', 
    # 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats', 'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 
    # 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'OVA']]
    # dropped.columns = list(map(normit, dropped.columns))
    
    df.columns = list(map(normit, df.columns))
    # dropped = df[[
    #     'age', 'growth', 'value', 'wage', 'reactions', 'power', 'shot_power', 'vision', 
    #     'composure', 'total_stats', 'base_stats', 'pas', 'dri', 'phy', 
    #     'ova']]
    dropped = df[['age', 'bp', 'height', 'weight', 'foot', 'growth', 'value', 'wage',
       'release_clause', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w_f', 'sm', 'a_w', 'd_w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'ova']]

    dropped['composure'] = dropped['composure'].fillna(np.mean(dropped['composure'])).copy()
    dropped['a_w'] = dropped['a_w'].fillna('Medium').copy()
    dropped['d_w'] = dropped['d_w'].fillna('Medium').copy()
    dropped = dropped.dropna(subset=['vision'])
    dropped['height'] = dropped['height'].apply(get_height)
    dropped['weight_lbs'] = dropped['weight'].str.rstrip("lbs").astype(int)
    dropped = dropped.drop(['weight'], axis=1)
    dropped['value'] = dropped['value'].apply(get_value)
    dropped['wage'] = dropped['wage'].apply(get_value)
    dropped['release_clause'] = dropped['release_clause'].apply(get_value) 
    dropped['w_f'] = dropped['w_f'].apply(get_stars)
    dropped['sm'] = dropped['sm'].apply(get_stars)
    dropped['ir'] = dropped['ir'].apply(get_stars)

    return dropped


In [11]:
def preprocess(df: pd.DataFrame):
    cleaned = clean_fifa_df(df)
    y = cleaned['ova']
    cleaned = cleaned.drop(['ova'], axis=1)
    nums = cleaned.select_dtypes(include=np.number)
    cats = cleaned.select_dtypes(include=object)
    nums = minmax(nums)
    cats = onehot(cats)
    X = pd.concat((nums, cats), axis=1)
    return X, y

In [20]:
validate = pd.read_csv('fifa21_validate.csv')
train = pd.read_csv('fifa21_train.csv')
validate.columns == train.columns

pd.DataFrame(validate.isna().sum()).to_csv('v.csv')

In [13]:
X, y = preprocess(train)
X2, y2 = preprocess(validate)

X.columns == X2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
lm = linear_model.LinearRegression()
lm.fit(X_train, y_train)

preds_test = lm.predict(X_test)
print(r2_score(y_test, preds_test))

preds = lm.predict(X2)
print(r2_score(y2, preds))

0.9178470459795224
-1.1629065325924342e+22


In [33]:
pd.set_option('display.max_columns', None)
display(X.describe())
X2.describe()

,age,height,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w_f,sm,ir,pac,sho,pas,dri,def,phy,weight_lbs,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right,a_w_low,a_w_medium,d_w_low,d_w_medium
count,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000,11660.000000
mean,0.342764,0.517406,0.212719,0.028364,0.018699,0.027284,0.548227,0.517899,0.489386,0.550679,0.595469,0.477527,0.523588,0.580616,0.506865,0.441723,0.538128,0.611014,0.598409,0.624090,0.640431,0.616855,0.547744,0.596367,0.544247,0.574843,0.571952,0.602506,0.586550,0.501497,0.576066,0.550951,0.491352,0.542424,0.534579,0.490992,0.570806,0.489220,0.494001,0.500519,0.474139,0.153913,0.154783,0.156890,0.155190,0.148540,0.156179,0.572021,0.507836,0.495626,0.364473,0.035913,0.601328,0.500351,0.498296,0.540494,0.493757,0.572983,0.385509,0.185506,0.071612,0.004117,0.056175,0.092539,0.054202,0.047942,0.012607,0.014751,0.054031,0.083105,0.019554,0.014151,0.158233,0.754288,0.051458,0.644254,0.108491,0.717667
std,0.183235,0.136114,0.223634,0.058607,0.037936,0.060889,0.183025,0.203022,0.210708,0.192438,0.168188,0.206197,0.180088,0.198344,0.202005,0.196321,0.177198,0.174448,0.156505,0.175550,0.170915,0.176638,0.126859,0.176641,0.159486,0.162523,0.171673,0.180683,0.165730,0.211648,0.175007,0.196121,0.244604,0.204887,0.161121,0.182290,0.138680,0.249350,0.226732,0.255578,0.252584,0.190433,0.191596,0.187933,0.183853,0.180414,0.195029,0.165459,0.153570,0.168945,0.194739,0.109980,0.160745,0.182067,0.149435,0.146387,0.218673,0.147746,0.125570,0.388724,0.257856,0.064032,0.230269,0.289797,0.226426,0.213652,0.111577,0.120561,0.226088,0.276052,0.138468,0.118118,0.364975,0.430527,0.220939,0.478759,0.311012,0.450154
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.185185,0.400000,0.000000,0.004167,0.001786,0.003026,0.481013,0.397727,0.326087,0.465909,0.542169,0.325581,0.442623,0.527473,0.366667,0.303371,0.428571,0.571429,0.518519,

,age,height,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w_f,sm,ir,pac,sho,pas,dri,def,phy,weight_lbs,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right,a_w_low,a_w_medium,d_w_low,d_w_medium
count,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000,1996.000000
mean,0.302056,0.489746,0.214208,0.033174,0.042005,0.029403,0.586497,0.548664,0.502373,0.532259,0.609559,0.500216,0.562537,0.602210,0.503886,0.436711,0.551885,0.621354,0.593739,0.642073,0.645996,0.612567,0.524352,0.593674,0.593854,0.579260,0.609098,0.629663,0.595028,0.527649,0.561652,0.558546,0.516324,0.550008,0.563027,0.507468,0.596973,0.474106,0.477421,0.492467,0.466237,0.153744,0.156808,0.153457,0.160909,0.158049,0.161192,0.579681,0.453835,0.497871,0.358843,0.052104,0.596842,0.487816,0.471255,0.562576,0.489010,0.563144,0.434819,0.190882,0.076653,0.005010,0.062625,0.100200,0.051603,0.049098,0.011523,0.013527,0.045591,0.082164,0.020541,0.015030,0.144790,0.747495,0.041082,0.660822,0.097194,0.735972
std,0.156679,0.177948,0.228460,0.069928,0.080397,0.067601,0.205342,0.217742,0.224012,0.203847,0.172982,0.221247,0.206410,0.211254,0.214057,0.207915,0.186720,0.201293,0.160424,0.175942,0.176443,0.188040,0.147244,0.186191,0.175390,0.177240,0.165058,0.186418,0.183720,0.226417,0.177385,0.203955,0.247966,0.223064,0.169869,0.186880,0.144252,0.249169,0.226793,0.262582,0.256840,0.201650,0.209489,0.187501,0.179209,0.195339,0.204565,0.172900,0.160958,0.169110,0.194268,0.155692,0.168869,0.191342,0.157392,0.160829,0.216227,0.168538,0.145357,0.393095,0.266107,0.070622,0.242348,0.300342,0.221280,0.216127,0.106752,0.115545,0.208649,0.274684,0.141878,0.121703,0.351977,0.434558,0.198530,0.473549,0.296297,0.440925
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.161290,0.383333,0.000000,0.004808,0.004000,0.003413,0.509804,0.418675,0.321839,0.435294,0.555556,0.345679,0.470395,0.540230,0.360465,0.290698,0.437500,0.567901,0.511834,0.554217,0.567901,0.513158,0.442623,0.480000,0.483871,0.445946,0.507042,0.5301

In [7]:
### pickle test

# X, y = preprocess(validate)

# model = pickle.load(open('model.sav', 'rb'))

# preds = model.predict(X)
# print(r2_score(y, preds))